In [ ]:

from rbot import Session
from rbot import Runner
from rbot import BinanceConfig
from rbot import BinanceMarket
from rbot import init_debug_log
from rbot import init_log
from rbot import OrderSide
from rbot import time_string
from rbot import OrderStatus
from rbot import SEC


In [ ]:
MARGIN = 0.05

class MyAgent:
    def __init__(self):
        self.last_update = 0
        self.order_size = 0.001

    def on_clock(self, session, clock):
        if 0 < len(session.buy_orders):
            if session.current_time + SEC(60) < session.buy_orders[0].create_time:
                session.cancel_order(session.buy_orders[0].order_id)
                print("cancel buy order")
        
        if 0 < len(session.sell_orders):
            if session.current_time + SEC(60) < session.sell_orders[0].create_time:
                session.cancel_order(session.sell_orders[0].order_id)
                print("cancel sell order")

    def on_tick(self, session, side, price, size):
        if session.buy_order_amount == 0 and ((- self.order_size * session.buy_edge) < session.home_change):
            session.limit_order(OrderSide.Buy, session.buy_edge - MARGIN, self.order_size)
            #print("try make buy order", session.home_change, session.foreign_change)
            
        if session.sell_order_amount == 0 and ((- self.order_size) < session.foreign_change):
            session.limit_order(OrderSide.Sell, session.sell_edge + MARGIN, self.order_size)
            #print("try make sell order", session.home_change, session.foreign_change)
    
#    def on_update(self, session, updated_order):
#        print("on_update", updated_order)
        
    def on_account_update(self, session, account):
        print("account update: ", session.current_time, account)

    




In [ ]:
market = BinanceMarket(BinanceConfig.BTCUSDT)

agent = MyAgent()
runner = Runner()

session = runner.back_test(market, agent, 60, 0, 0)

session

In [ ]:
session.log_df[session.log_df["status"] == "Canceled"]

In [ ]:
df = session.log_df.to_pandas(use_pyarrow_extension_array=True)
df


In [ ]:
df.to_csv("backtest.csv")

In [ ]:
df['status'].value_counts()

In [ ]:
df['foreign_change'].cumsum()

In [ ]:
session.log_df.sum()

In [ ]:
session.home_change + session.foreign_change * session.buy_edge

In [ ]:
len(session.log_df)